### Modelling memory distortions with a variational autoencoder: DRM experiment using news articles dataset

The Deese-Roediger-McDermott task is a classic way to measure memory distortion. This notebook tries to recreate the human results in VAE and AE models.

Steps:
* Process dataset of CNN / Daily Mail news articles (https://www.tensorflow.org/datasets/catalog/cnn_dailymail) into lists of words (ignoring order)
* Vectorize these (into vectors of word counts, removing most common and least common to reduce dimension)
* Train VAE and normal AE to reconstruct word vectors
* Explore whether we see 'intrusion of semantically related items' when network recalls a list
* Use the word lists at https://www3.nd.edu/~memory/OLD/Materials/DRM.pdf to test this - are the lure words falsely recalled?
* Explore what generating word lists from latent space tells us about the 'semantic memory' of the network

In [ ]:
!pip install tensorflow==2.2.0
!pip install tensorflow_datasets

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#### Load data and preprocess

In [2]:
ds = tfds.load('cnn_dailymail', split='train')

articles = []
for example in ds: 
    articles.append(example["article"].numpy().decode("utf-8"))
    
print(len(articles))

287113


In [4]:
texts = articles[0:100000]

The cell below vectorizes the articles - it turns 'word1 word2 word3' into a vector with 1 at index for word1, 1 at index for word2, and 1 at index for word3. vectorizer.vocabulary_ stores the mapping of words to indices.

To make the vocabulary manageable, I filter out words in greater than 20% or fewer than 1% of documents:

In [5]:
vectorizer = CountVectorizer(max_df=0.2, min_df=0.01)
X = vectorizer.fit_transform(texts)
print(len(vectorizer.get_feature_names()))

4316


In [7]:
x_train = X.toarray()
x_train[0]

array([0, 0, 0, ..., 0, 0, 0])

The inverse_transform function reverses the vectorisation:

In [8]:
vectorizer.inverse_transform(X[150])

[array(['14', '2013', '15', 'late', 'anyone', 'alert', 'murder', 'known',
        'pose', 'released', 'woman', 'death', 'white', 'contact', 'danger',
        'able', 'although', 'following', 'hope', 'arrest', 'current',
        'danny', 'centre', 'wife', 'husband', 'area', 'fear', 'asked',
        'public', 'town', 'incident', 'looking', 'spokesman', 'wanted',
        'flat', 'jane', 'images', 'call', 'launched', 'connection', '19',
        'approach', 'leave', 'ex', 'threat', 'murdered', 'body', 'dead',
        '18', 'considered', 'ms', 'regular', 'august', 'close', 'hair',
        'keen', 'friday', 'officers', 'build', '08', 'discovered', 'knew',
        'described', 'knows', 'wish', 'hunting', 'force', 'track',
        'appears', 'warn', 'inquiry', '57', '34', '02', 'grey', 'frequent'],
       dtype='<U15')]

#### Function to build VAE:

In [39]:
# Set the dimension of the latent space, i.e. number of latent variables
latent_dim = 1000
input_dim = len(vectorizer.get_feature_names())

## Create a sampling layer

class Sampling(layers.Layer):
    # Uses (z_mean, z_log_var) to sample z, the vector encoding a digit.

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## Build the encoder

encoder_inputs = keras.Input(shape=(input_dim,))
z_mean = layers.Dense(latent_dim, name="z_mean")(encoder_inputs)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(encoder_inputs)
# This uses the special sampling layer defined above:
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
# Prints the structure of the model below:
encoder.summary()

## Build the decoder

latent_inputs = keras.Input(shape=(latent_dim,))
decoder_outputs = layers.Dense(input_dim, activation="relu")(latent_inputs)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
# Prints the structure of the model below:
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4316)]       0                                            
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 1000)         4317000     input_3[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 1000)         4317000     input_3[0][0]                    
__________________________________________________________________________________________________
sampling_1 (Sampling)           (None, 1000)         0           z_mean[0][0]                     
                                                                 z_log_var[0][0]            

In [40]:
## Define the VAE as a `Model` with a custom `train_step` 
# In inherits from the keras Model class, giving it all the properties of a usual keras model

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= input_dim
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + 0.01*kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }


In [41]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(x_train, epochs=20, batch_size=64)

Epoch 1/20
1563/1563 [==============================] - 6s 4ms/step - loss: 3398.6217 - reconstruction_loss: 3398.6018 - kl_loss: 1.9910
Epoch 2/20
1563/1563 [==============================] - 5s 4ms/step - loss: 61988241.2188 - reconstruction_loss: 3364.6348 - kl_loss: 6198488120.5482
Epoch 3/20
1563/1563 [==============================] - 5s 4ms/step - loss: 12371.8121 - reconstruction_loss: 3425.6717 - kl_loss: 894614.1119
Epoch 4/20
1563/1563 [==============================] - 5s 4ms/step - loss: 6056.1815 - reconstruction_loss: 3412.3722 - kl_loss: 264380.9510
Epoch 5/20
1563/1563 [==============================] - 5s 4ms/step - loss: 430932.1496 - reconstruction_loss: 3349.4788 - kl_loss: 42758267.2040
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 292438.5698 - reconstruction_loss: 3324.1277 - kl_loss: 28911445.3864
Epoch 7/20
1563/1563 [==============================] - 5s 4ms/step - loss: 7159.1623 - reconstruction_loss: 3278.3963 - kl_loss: 388076

In [42]:
x_train[0]

array([0, 0, 0, ..., 0, 0, 0])

In [50]:
def recall_list(test_item):
    encoded = vae.encoder.predict(vectorizer.transform([test_item]))
    decoded = vae.decoder.predict(encoded)

    word_lookup = {v:k for k,v in vectorizer.vocabulary_.items()}

    for index in np.argsort(-decoded)[0][0:10]:
        print(word_lookup[index])

In [53]:
recall_list("health doctor hospital medical medicine treat")

health
medical
hospital
doctor
patients
doctors
hospitals
treatment
risk
disease


In [54]:
recall_list("usa states united president congress senate")

states
united
washington
american
president
leader
obama
office
federal
house


#### Load DRM word lists

Load subset of lists from https://www3.nd.edu/~memory/OLD/Materials/DRM.pdf

In [45]:
DRM_lists = []
lures = []

DRM_lists.append(['STEAL', 'ROBBER', 'JAIL', 'VILLAIN', 'BANDIT', 'CRIMINAL', 'ROB','COP', 'MONEY', 'BAD', 'BURGLAR', 'CROOK', 'CRIME', 'GUN', 'BANK'])
lures.append('THIEF')

DRM_lists.append(['CLINIC', 'HEALTH', 'MEDICINE', 'SICK', 'STETHOSCOPE', 'CURE', 'NURSE', 'SURGEON', 'PATIENT', 'HOSPITAL', 'DENTIST', 'PHYSICIAN', 'ILL'])
lures.append('DOCTOR')

DRM_lists.append(['CHILLY', 'HOT', 'WET', 'WINTER', 'FREEZE', 'FRIGID', 'HEAT', 'SNOW', 'ARCTIC', 'AIR', 'WEATHER', 'SHIVER', 'ICE', 'FROST', 'WARM'])
lures.append('COLD')

In [46]:
for ind, DRM_list in enumerate(DRM_lists):
    in_vocab = [i.lower() for i in DRM_list if i.lower() in vectorizer.vocabulary_.keys()]
    print("Words in DRM list for lure '{}':".format(lures[ind].lower()))
    print(in_vocab)
    test_item = ' '.join([i.lower() for i in DRM_list])
    encoded = vae.encoder.predict(vectorizer.transform([test_item]))
    decoded = vae.decoder.predict(encoded)
    print("Recalled list:")
    top_words = [word_lookup[index] for index in np.argsort(-decoded)[0][0:20]]
    print(top_words)
    print("...........")
    

Words in DRM list for lure 'thief':
['jail', 'criminal', 'rob', 'money', 'bad', 'crime', 'gun', 'bank']
Recalled list:
['bank', 'money', 'criminal', 'shot', '2013', 'accused', 'economic', 'london', 've', 'britain', 'tax', 'officers', 'public', 'head', 'court', 'leg', 'use', 'your', 'assault', 'men']
...........
Words in DRM list for lure 'doctor':
['clinic', 'health', 'medicine', 'sick', 'nurse', 'patient', 'hospital', 'ill']
Recalled list:
['health', 'hospital', 'doctors', 'hospitals', 'patients', 'treatment', 'doctor', 'hours', 'risk', 'medical', 'disease', 'care', 'countries', 'reported', 'africa', 'general', 'condition', 'women', 'cases', 'baby']
...........
Words in DRM list for lure 'cold':
['hot', 'wet', 'winter', 'heat', 'snow', 'air', 'weather', 'ice', 'warm']
Recalled list:
['north', 'weather', 'across', 'northern', 'west', 'cold', 'change', 'office', 'london', 'water', '2013', 'scotland', 'rain', 'britain', 'park', 'great', 'areas', 'british', 'use', 'space']
...........


### Basic autoencoder for comparison

Compare VAE with standard AE.

In [58]:
num_words=4316

input_layer = keras.Input(shape=(num_words,))
encoded = layers.Dense(1000, activation='relu')(input_layer)
decoded = layers.Dense(num_words, activation='sigmoid')(encoded)

autoencoder = keras.Model(input_layer, decoded)

encoder = keras.Model(input_layer, encoded)

encoded_input = keras.Input(shape=(1000,))
decoder_layer = autoencoder.layers[-1]
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
               epochs=10,
               batch_size=256,
               shuffle=True)

Epoch 1/10
391/391 [==============================] - 3s 7ms/step - loss: -0.8083
Epoch 2/10
391/391 [==============================] - 3s 7ms/step - loss: -12.7520
Epoch 3/10
391/391 [==============================] - 3s 7ms/step - loss: -40.5550
Epoch 4/10
391/391 [==============================] - 3s 7ms/step - loss: -81.4714
Epoch 5/10
391/391 [==============================] - 3s 7ms/step - loss: -133.7468
Epoch 6/10
391/391 [==============================] - 3s 7ms/step - loss: -196.2966
Epoch 7/10
391/391 [==============================] - 3s 7ms/step - loss: -268.2217
Epoch 8/10
391/391 [==============================] - 3s 7ms/step - loss: -348.9727
Epoch 9/10
391/391 [==============================] - 3s 7ms/step - loss: -437.6929
Epoch 10/10
391/391 [==============================] - 3s 7ms/step - loss: -534.3267


In [73]:
test_item = "usa america american states united president congress senate"

encoded = encoder.predict(vectorizer.transform([test_item]))
decoded = decoder.predict(encoded)

In [74]:
word_lookup = {v:k for k,v in vectorizer.vocabulary_.items()}

for index in np.argsort(-decoded)[0][0:20]:
    print(word_lookup[index])

administration
sanctions
russian
russia
gop
government
bush
group
health
campaign
troops
republican
bin
house
bill
syria
ukraine
barack
united
republicans


In [76]:
for ind, DRM_list in enumerate(DRM_lists):
    in_vocab = [i.lower() for i in DRM_list if i.lower() in vectorizer.vocabulary_.keys()]
    print("Words in DRM list for lure '{}':".format(lures[ind].lower()))
    print(in_vocab)
    test_item = ' '.join([i.lower() for i in DRM_list])
    encoded = encoder.predict(vectorizer.transform([test_item]))
    decoded = decoder.predict(encoded)
    print("Recalled list:")
    top_words = [word_lookup[index] for index in np.argsort(-decoded)[0][0:20]]
    print(top_words)
    print("...........")
    

Words in DRM list for lure 'thief':
['jail', 'criminal', 'rob', 'money', 'bad', 'crime', 'gun', 'bank']
Recalled list:
['animal', 'financial', 'money', 'shooting', 'animals', 'murder', 'death', 'food', 'property', 'judge', 'ford', 'company', 'million', 'fraud', 'prison', 'car', 'drugs', 'drug', 'guns', 'case']
...........
Words in DRM list for lure 'doctor':
['clinic', 'health', 'medicine', 'sick', 'nurse', 'patient', 'hospital', 'ill']
Recalled list:
['officials', 'africa', 'blood', 'per', 'patients', 'patient', 'passengers', 'hospitals', 'nurse', 'nhs', 'ms', 'food', 'cancer', 'mrs', 'mother', 'care', 'cases', 'medical', 'cent', 'treatment']
...........
Words in DRM list for lure 'cold':
['hot', 'wet', 'winter', 'heat', 'snow', 'air', 'weather', 'ice', 'warm']
Recalled list:
['service', 'winter', 'uk', 'air', 'aircraft', 'tuesday', 'airlines', 'airport', 'car', 'expected', 'power', 'climate', 'woman', 'frozen', 'lake', 'county', 'london', 'heavy', 'hit', 'winds']
...........
